<a href="https://colab.research.google.com/github/nananair/Research-NLP-projects/blob/main/Quartile_based_distributional_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook performs quartile-based comparison to explore which discourse features (news values, rhetorical strategies, etc.) are associated with higher visibility (social media shares, applause, citations, media coverage, etc.).

Adaptable to multiple contexts:
- Social media engagement (Twitter/Facebook shares, likes, comments, retweets)
- Parliamentary discourse (applause or laughter counts, interruptions)
- Academic impact (citation counts, downloads)
- Media coverage (headline counts, mentions)
- Any count-based visibility metric

How it works:
- Divides your data into quartiles (Q1=lowest visibility, Q4=highest visibility)
- Calculates mean discourse feature values in each quartile
- Identifies which features are most prevalent
- Provides statistical tests and visualisations

No coding required:
Simply upload your dataset (Excel or CSV) and run all cells.

Dataset requirements:
- At least one column with count data (e.g., Share_counts, Applause_counts)
- Columns for each discourse feature you want to analyse
- Optional: A "Dataset" column if analysing multiple cases/contexts

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [3]:
print("=" * 70)
print("QUARTILE-BASED DISTRIBUTIONAL ANALYSIS")
print("=" * 70)
print("Please upload your labelled dataset (Excel or CSV format)")
print("Your dataset should include:")
print("Count columns (e.g., Share_counts_Twitter, Applause_counts)")
print("Discourse feature columns (e.g., Eliteness, Positivity)")
print("Optional: Dataset column for multi-case analysis")
print("=" * 70 + "\n")

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".xlsx"):
        df = pd.read_excel(filename)
    elif filename.endswith(".csv"):
        df = pd.read_csv(filename)
    else:
        raise ValueError("File must be .xlsx or .csv")
    break

print(f"Dataset loaded successfully: {len(df)} rows, {len(df.columns)} columns")
print("\nFirst 5 rows:")
display(df.head())

# Clean column names
df.columns = df.columns.str.strip().str.replace(r"\s+", "_", regex=True)

QUARTILE-BASED DISTRIBUTIONAL ANALYSIS
Please upload your labelled dataset (Excel or CSV format)
Your dataset should include:
Count columns (e.g., Share_counts_Twitter, Applause_counts)
Discourse feature columns (e.g., Eliteness, Positivity)
Optional: Dataset column for multi-case analysis



Saving updated_thesis_dataset_v2.xlsx to updated_thesis_dataset_v2.xlsx
Dataset loaded successfully: 192 rows, 15 columns

First 5 rows:


,Articles,Outlep_type,Word_length,Share_counts_Twitter,Share_counts_Facebook,Dataset,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Space,Positivity
0,Second police officer died by suicide followin...,Regional,247.0,21800,21200,USA,4,0,5,6,1,0,0,8,0
1,Rioters breached US Capitol security on Wednes...,Regional,622.0,19800,24100,USA,11,4,8,9,10,0,2,11,0
2,‘They Got a Officer!’: How a Mob Dragged and B...,Regional,441.0,12900,23400,USA,2,0,9,15,0,0,0,9,0
3,"As the D.C. police clear the Capitol grounds, ...",Regional,468.0,16400,20100,USA,7,0,2,4,7,0,2,13,0
4,Now it’s sinking in: Wednesday’s Capitol Hill ...,Regional,1042.0,17200,23700,USA,9,9,23,10,8,11,2,11,0


In [4]:
print("\n" + "=" * 70)
print("CONFIGURATION")
print("=" * 70)

CONFIG = {
    # CASE/GROUPING VARIABLE
    # Set to None if analysing a single dataset
    "CASE_COL": "Dataset",  # Column name for grouping (e.g., country, time period)

    # VISIBILITY METRICS (count columns to analyse)
    # The code will automatically detect which of these exist in your data
    "VISIBILITY_METRICS": [
        "Share_counts",           # Single platform
        "Share_counts_Twitter",   # Twitter-specific
        "Share_counts_Facebook",  # Facebook-specific
        "Applause_counts",        # Parliamentary data
        "Citation_counts",        # Academic impact
        "Headline_counts",        # Media coverage
        # Add your own metrics here
    ],

    # DISCOURSE FEATURES
    # Customise based on your analytical framework
    "DISCOURSE_FEATURES": [
        "Eliteness", "Personalisation", "Superlativeness", "Negativity",
        "Timeliness", "Unexpectedness", "Impact", "Proximity", "Positivity"
    ],

    # Alternative naming for proximity
    "RENAME_FEATURES": {
        "Space": "Proximity",
    },

    # ANALYSIS OPTIONS
    "QUARTILES_WITHIN_CASE": True,     # Calculate quartiles separately per case
    "CREATE_COMBINED_METRIC": True,    # Combine Twitter+Facebook if both exist
    "STATISTICAL_TESTS": True,         # Run Kruskal-Wallis tests
    "GENERATE_PLOTS": True,            # Create visualizations
    "EFFECT_SIZE_THRESHOLD": 0.5,      # Minimum difference to highlight (in standardized units)
}

# Apply feature renaming
if CONFIG["RENAME_FEATURES"]:
    df = df.rename(columns=CONFIG["RENAME_FEATURES"])
    print(f"✓ Renamed {len(CONFIG['RENAME_FEATURES'])} feature columns")


CONFIGURATION
✓ Renamed 1 feature columns


In [5]:
print("\n" + "=" * 70)
print("DATA PREPARATION")
print("=" * 70)

CASE_COL = CONFIG["CASE_COL"]
if CASE_COL is None or CASE_COL not in df.columns:
    df["Dataset"] = "All_articles"
    CASE_COL = "Dataset"
    print("✓ Single dataset mode: all articles treated as one group")
else:
    n_cases = df[CASE_COL].nunique()
    print(f"✓ Multi-case mode: {n_cases} groups detected")
    print(f"  Cases: {', '.join(df[CASE_COL].unique().astype(str))}")

available_metrics = [m for m in CONFIG["VISIBILITY_METRICS"] if m in df.columns]
if not available_metrics:
    raise ValueError("No visibility metrics found. Check CONFIG['VISIBILITY_METRICS']")

print(f"\n✓ Found {len(available_metrics)} visibility metric(s):")
for metric in available_metrics:
    n_valid = df[metric].notna().sum()
    print(f"  • {metric}: {n_valid} valid observations")

CREATE_COMBINED = (
    CONFIG["CREATE_COMBINED_METRIC"] and
    "Share_counts_Twitter" in available_metrics and
    "Share_counts_Facebook" in available_metrics
)

if CREATE_COMBINED:
    df["Share_counts_Combined"] = (
        pd.to_numeric(df["Share_counts_Twitter"], errors="coerce").fillna(0) +
        pd.to_numeric(df["Share_counts_Facebook"], errors="coerce").fillna(0)
    )
    available_metrics.append("Share_counts_Combined")
    print("✓ Created combined Twitter+Facebook metric")

DISCOURSE_FEATURES = CONFIG["DISCOURSE_FEATURES"]
for feat in DISCOURSE_FEATURES:
    if feat not in df.columns:
        df[feat] = 0
        print(f"Feature '{feat}' not found - created as zeros")

df[DISCOURSE_FEATURES] = df[DISCOURSE_FEATURES].apply(pd.to_numeric, errors="coerce").fillna(0)
print(f"\n✓ Prepared {len(DISCOURSE_FEATURES)} discourse features")


DATA PREPARATION
✓ Multi-case mode: 7 groups detected
  Cases: USA, Iraq, Peru, Chad, Chad , Kenya, Benin

✓ Found 2 visibility metric(s):
  • Share_counts_Twitter: 192 valid observations
  • Share_counts_Facebook: 192 valid observations
✓ Created combined Twitter+Facebook metric

✓ Prepared 9 discourse features


In [6]:
QUART_LABELS = ["Q1 (Low)", "Q2", "Q3", "Q4 (High)"]
quartile_cat = pd.CategoricalDtype(categories=QUART_LABELS, ordered=True)

def assign_quartiles(data, metric_col, within_cases=True):
    """
    Assign quartiles based on visibility metric.
    Handles ties and small samples gracefully.
    """
    tmp = data.copy()
    tmp["_metric_"] = pd.to_numeric(tmp[metric_col], errors="coerce")
    tmp = tmp[tmp["_metric_"].notna()].copy()

    if within_cases and CASE_COL in tmp.columns:
        parts = []
        for case, group in tmp.groupby(CASE_COL, dropna=False):
            try:
                group["Visibility_Quartile"] = pd.qcut(
                    group["_metric_"], q=4, labels=QUART_LABELS, duplicates='drop'
                )
            except ValueError:
                ranks = group["_metric_"].rank(method="average")
                group["Visibility_Quartile"] = pd.qcut(
                    ranks, q=4, labels=QUART_LABELS, duplicates='drop'
                )
            parts.append(group)
        tmp = pd.concat(parts, ignore_index=True)
    else:
        try:
            tmp["Visibility_Quartile"] = pd.qcut(
                tmp["_metric_"], q=4, labels=QUART_LABELS, duplicates='drop'
            )
        except ValueError:
            ranks = tmp["_metric_"].rank(method="average")
            tmp["Visibility_Quartile"] = pd.qcut(
                ranks, q=4, labels=QUART_LABELS, duplicates='drop'
            )

    return tmp

In [7]:
def build_comparison_table(data, metric_name, metric_col):
    tmp = assign_quartiles(data, metric_col, within_cases=CONFIG["QUARTILES_WITHIN_CASE"])

    grouped = (
        tmp.groupby([CASE_COL, "Visibility_Quartile"], observed=False)
        [DISCOURSE_FEATURES]
        .agg(['mean', 'count'])
        .reset_index()
    )

    grouped.columns = [CASE_COL, "Visibility_Quartile"] + [
        f"{feat}_{stat}" for feat in DISCOURSE_FEATURES for stat in ['mean', 'count']
    ]

    keep_cols = [CASE_COL, "Visibility_Quartile"] + \
                [f"{feat}_mean" for feat in DISCOURSE_FEATURES] + \
                [f"{DISCOURSE_FEATURES[0]}_count"]

    grouped = grouped[keep_cols].rename(
        columns={f"{DISCOURSE_FEATURES[0]}_count": "n_articles"}
    )

    rename_dict = {f"{feat}_mean": feat for feat in DISCOURSE_FEATURES}
    grouped = grouped.rename(columns=rename_dict)

    filled = []
    for case in grouped[CASE_COL].unique():
        case_data = grouped[grouped[CASE_COL] == case]
        idx = pd.MultiIndex.from_product(
            [[case], QUART_LABELS],
            names=[CASE_COL, "Visibility_Quartile"]
        )
        case_data = case_data.set_index([CASE_COL, "Visibility_Quartile"]).reindex(idx)
        case_data[DISCOURSE_FEATURES] = case_data[DISCOURSE_FEATURES].fillna(0)
        case_data["n_articles"] = case_data["n_articles"].fillna(0).astype(int)
        filled.append(case_data.reset_index())

    result = pd.concat(filled, ignore_index=True)
    result.insert(1, "Visibility_Metric", metric_name)

    result[DISCOURSE_FEATURES] = result[DISCOURSE_FEATURES].round(3)

    return result

print("\n" + "=" * 70)
print("BUILDING QUARTILE COMPARISON TABLES")
print("=" * 70)

all_tables = []
for metric in available_metrics:
    print(f"\nProcessing: {metric}")
    table = build_comparison_table(df, metric, metric)
    all_tables.append(table)
    print(f"  ✓ Created comparison table ({len(table)} rows)")

final_table = pd.concat(all_tables, ignore_index=True)
print(f"Generated {len(all_tables)} comparison table(s)")


BUILDING QUARTILE COMPARISON TABLES

Processing: Share_counts_Twitter
  ✓ Created comparison table (28 rows)

Processing: Share_counts_Facebook
  ✓ Created comparison table (28 rows)

Processing: Share_counts_Combined
  ✓ Created comparison table (28 rows)
Generated 3 comparison table(s)


In [8]:
def kruskal_wallis_test(data, metric_name, metric_col):
    tmp = assign_quartiles(data, metric_col, within_cases=False)

    results = []
    for feature in DISCOURSE_FEATURES:
        groups = [
            group[feature].values
            for name, group in tmp.groupby("Visibility_Quartile", observed=True)
            if len(group[feature].dropna()) > 0
        ]

        if len(groups) >= 2:
            h_stat, p_value = stats.kruskal(*groups)

            n = len(tmp)
            eta_sq = (h_stat - len(groups) + 1) / (n - len(groups))

            results.append({
                "Visibility_Metric": metric_name,
                "Feature": feature,
                "H_statistic": h_stat,
                "p_value": p_value,
                "eta_squared": eta_sq,
                "significance": "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
            })

    return pd.DataFrame(results)

if CONFIG["STATISTICAL_TESTS"]:
    print("\n" + "=" * 70)
    print("STATISTICAL TESTING (Kruskal-Wallis H-Test)")
    print("=" * 70)
    print("\nTesting whether discourse features differ significantly across quartiles...")

    all_tests = []
    for metric in available_metrics:
        print(f"\n• {metric}:")
        test_results = kruskal_wallis_test(df, metric, metric)
        all_tests.append(test_results)

        sig_results = test_results[test_results["p_value"] < 0.05].sort_values("p_value")
        if len(sig_results) > 0:
            print(f"  Found {len(sig_results)} significant feature(s):")
            for _, row in sig_results.iterrows():
                print(f"    {row['Feature']}: p={row['p_value']:.4f} {row['significance']}, η²={row['eta_squared']:.3f}")
        else:
            print("  No significant differences found")

    statistical_tests = pd.concat(all_tests, ignore_index=True)
    print("Statistical testing complete")


STATISTICAL TESTING (Kruskal-Wallis H-Test)

Testing whether discourse features differ significantly across quartiles...

• Share_counts_Twitter:
  Found 7 significant feature(s):
    Unexpectedness: p=0.0000 ***, η²=0.169
    Superlativeness: p=0.0000 ***, η²=0.140
    Impact: p=0.0000 ***, η²=0.111
    Personalisation: p=0.0001 ***, η²=0.093
    Negativity: p=0.0023 **, η²=0.061
    Proximity: p=0.0034 **, η²=0.057
    Positivity: p=0.0477 *, η²=0.026

• Share_counts_Facebook:
  Found 5 significant feature(s):
    Unexpectedness: p=0.0000 ***, η²=0.174
    Personalisation: p=0.0000 ***, η²=0.107
    Negativity: p=0.0003 ***, η²=0.083
    Superlativeness: p=0.0015 **, η²=0.066
    Proximity: p=0.0401 *, η²=0.028

• Share_counts_Combined:
  Found 5 significant feature(s):
    Unexpectedness: p=0.0000 ***, η²=0.174
    Negativity: p=0.0001 ***, η²=0.097
    Superlativeness: p=0.0002 ***, η²=0.088
    Personalisation: p=0.0013 **, η²=0.067
    Proximity: p=0.0079 **, η²=0.047
Statistica

In [9]:
def identify_key_patterns(table):
    patterns = []

    for metric in table["Visibility_Metric"].unique():
        metric_data = table[table["Visibility_Metric"] == metric]

        for case in metric_data[CASE_COL].unique():
            case_data = metric_data[metric_data[CASE_COL] == case]

            q1 = case_data[case_data["Visibility_Quartile"] == "Q1 (Low)"][DISCOURSE_FEATURES].values
            q4 = case_data[case_data["Visibility_Quartile"] == "Q4 (High)"][DISCOURSE_FEATURES].values

            if len(q1) > 0 and len(q4) > 0:
                differences = q4[0] - q1[0]

                for i, feature in enumerate(DISCOURSE_FEATURES):
                    diff = differences[i]
                    if abs(diff) >= CONFIG["EFFECT_SIZE_THRESHOLD"]:
                        patterns.append({
                            "Visibility_Metric": metric,
                            "Case": case,
                            "Feature": feature,
                            "Q1_mean": q1[0][i],
                            "Q4_mean": q4[0][i],
                            "Difference": diff,
                            "Direction": "Higher in Q4" if diff > 0 else "Lower in Q4"
                        })

    return pd.DataFrame(patterns)

print("\n" + "=" * 70)
print("KEY PATTERNS (Q4 vs Q1 Differences)")
print("=" * 70)

patterns = identify_key_patterns(final_table)
if len(patterns) > 0:
    patterns = patterns.sort_values("Difference", key=abs, ascending=False)
    print(f"\nFound {len(patterns)} notable pattern(s) (|difference| ≥ {CONFIG['EFFECT_SIZE_THRESHOLD']}):\n")
    display(patterns)
else:
    print("\nNo large differences detected between Q1 and Q4")
    print("(Consider lowering EFFECT_SIZE_THRESHOLD in CONFIG)")


KEY PATTERNS (Q4 vs Q1 Differences)

Found 101 notable pattern(s) (|difference| ≥ 0.5):



,Visibility_Metric,Case,Feature,Q1_mean,Q4_mean,Difference,Direction
26,Share_counts_Twitter,USA,Eliteness,3.000,54.600,51.600,Higher in Q4
93,Share_counts_Combined,USA,Eliteness,7.000,54.200,47.200,Higher in Q4
61,Share_counts_Facebook,USA,Eliteness,8.000,54.200,46.200,Higher in Q4
100,Share_counts_Combined,USA,Proximity,10.400,36.800,26.400,Higher in Q4
68,Share_counts_Facebook,USA,Proximity,11.200,36.800,25.600,Higher in Q4
...,...,...,...,...,...,...,...
7,Share_counts_Twitter,Chad,Personalisation,0.938,0.429,-0.509,Lower in Q4
54,Share_counts_Facebook,Peru,Personalisation,2.000,2.500,0.500,Higher in Q4
49,Share_counts_Facebook,Kenya,Negativity,9.000,8.500,-0.500,Lower in Q4
50,Share_counts_Facebook,Kenya,Timeliness,3.167,2.667,-0.500,Lower in Q4


In [10]:
print("\n" + "=" * 70)
print("QUARTILE COMPARISON RESULTS")
print("=" * 70)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

for metric in final_table["Visibility_Metric"].unique():
    print(f"\n{'=' * 70}")
    print(f"VISIBILITY METRIC: {metric}")
    print(f"{'=' * 70}")

    metric_data = final_table[final_table["Visibility_Metric"] == metric]

    for case in metric_data[CASE_COL].unique():
        print(f"\n--- Case: {case} ---\n")
        case_table = metric_data[metric_data[CASE_COL] == case]

        display_table = case_table.set_index("Visibility_Quartile")[
            ["n_articles"] + DISCOURSE_FEATURES
        ]
        display(display_table)


QUARTILE COMPARISON RESULTS

VISIBILITY METRIC: Share_counts_Twitter

--- Case: Benin ---



,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Visibility_Quartile,,,,,,,,,,
Q1 (Low),7,12.286,1.857,0.714,5.286,2.857,0.0,0.0,8.143,0.0
Q2,3,10.000,1.000,1.667,7.667,3.000,0.0,0.0,5.000,0.0
Q3,5,15.400,1.000,1.200,6.000,3.000,0.0,0.0,8.600,0.0
Q4 (High),5,13.200,0.800,2.800,10.600,5.200,0.0,0.0,9.400,0.0



--- Case: Chad ---



,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Visibility_Quartile,,,,,,,,,,
Q1 (Low),16,8.188,0.938,0.0,5.188,3.125,0.0,0.0,6.250,0.0
Q2,14,8.071,0.857,0.0,5.500,4.357,0.0,0.0,8.214,0.0
Q3,14,6.500,0.500,0.0,4.286,3.929,0.0,0.0,5.786,0.0
Q4 (High),14,6.857,0.429,0.0,5.714,2.500,0.0,0.0,6.000,0.0



--- Case: Chad  ---



...

,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Visibility_Quartile,,,,,,,,,,
Q1 (Low),7,26.286,0.429,6.0,9.714,3.0,0.0,10.286,11.429,2.571
Q2,4,22.000,2.250,6.0,7.500,2.5,0.0,8.750,9.500,2.250
Q3,5,20.400,1.800,5.6,8.200,2.6,0.0,8.800,9.800,1.800
Q4 (High),6,23.833,0.667,6.5,9.000,3.0,0.0,6.500,10.333,2.000



--- Case: Peru ---



,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Visibility_Quartile,,,,,,,,,,
Q1 (Low),4,25.500,0.75,2.00,4.500,6.000,0.0,3.25,6.500,2.000
Q2,4,16.750,1.25,1.75,3.250,3.250,0.0,2.00,5.500,2.500
Q3,3,29.667,2.00,1.00,3.667,5.333,0.0,4.00,8.333,2.667
Q4 (High),4,29.000,2.50,3.00,3.750,7.000,0.0,6.50,7.750,2.750



--- Case: USA ---



,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Visibility_Quartile,,,,,,,,,,
Q1 (Low),5,7.0,2.2,7.4,14.4,2.4,0.60,2.6,10.40,0.0
Q2,5,9.0,4.0,9.6,9.0,7.4,2.40,2.0,14.60,0.0
Q3,4,30.5,6.5,8.5,9.5,2.0,2.25,4.0,13.25,0.0
Q4 (High),5,54.2,21.8,23.6,29.2,16.2,3.00,13.6,36.80,0.0


In [11]:
print("\n" + "=" * 70)
print("EXPORTING RESULTS")
print("=" * 70)

output_file = "quartile_analysis_results.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    final_table.to_excel(writer, sheet_name="All_Results", index=False)

    if CONFIG["STATISTICAL_TESTS"]:
        statistical_tests.to_excel(writer, sheet_name="Statistical_Tests", index=False)

    if len(patterns) > 0:
        patterns.to_excel(writer, sheet_name="Key_Patterns", index=False)

    for metric in available_metrics:
        metric_data = final_table[final_table["Visibility_Metric"] == metric]
        for case in metric_data[CASE_COL].unique():
            case_data = metric_data[metric_data[CASE_COL] == case]
            sheet_name = f"{metric}_{case}"[:31]  # Excel 31-char limit
            case_data[["Visibility_Quartile", "n_articles"] + DISCOURSE_FEATURES].to_excel(
                writer, sheet_name=sheet_name, index=False
            )

    guide = pd.DataFrame({
        "Interpretation Guide": [
            "QUARTILE ANALYSIS INTERPRETATION",
            "",
            "Q1 (Low): Bottom 25% of visibility",
            "Q4 (High): Top 25% of visibility",
            "",
            "How to interpret:",
            "- Higher values in Q4 = feature associated with MORE visibility",
            "- Lower values in Q4 = feature associated with LESS visibility",
            "- Similar values across quartiles = feature not associated with visibility",
            "",
            "Statistical significance (p-values):",
            "* p < 0.05  (significant)",
            "** p < 0.01  (highly significant)",
            "*** p < 0.001  (very highly significant)",
            "",
            "Effect size (eta-squared):",
            "0.01 = small effect",
            "0.06 = medium effect",
            "0.14 = large effect",
            "",
            "REPORTING RECOMMENDATIONS:",
            "1. Report quartile means and sample sizes",
            "2. Include statistical test results (H-statistic, p-value)",
            "3. Interpret substantive differences (Q4 vs Q1)",
            "4. Consider contextual factors when patterns differ across cases"
        ]
    })
    guide.to_excel(writer, sheet_name="Interpretation_Guide", index=False)

print(f"Results exported to: {output_file}")
print("\nExported sheets:")
print("  • All_Results: Complete quartile comparison data")
if CONFIG["STATISTICAL_TESTS"]:
    print("  • Statistical_Tests: Kruskal-Wallis H-test results")
if len(patterns) > 0:
    print("  • Key_Patterns: Notable Q1 vs Q4 differences")
print("  • Individual sheets for each metric-case combination")
print("  • Interpretation_Guide: How to read and report results")

print("Downloading results file...")
files.download(output_file)



EXPORTING RESULTS
Results exported to: quartile_analysis_results.xlsx

Exported sheets:
  • All_Results: Complete quartile comparison data
  • Statistical_Tests: Kruskal-Wallis H-test results
  • Key_Patterns: Notable Q1 vs Q4 differences
  • Individual sheets for each metric-case combination
  • Interpretation_Guide: How to read and report results


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
print("\n" + "=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)

print("SUMMARY:")
print(f"  • Analysed {len(available_metrics)} visibility metric(s)")
print(f"  • Examined {len(DISCOURSE_FEATURES)} discourse features")
print(f"  • Across {df[CASE_COL].nunique()} case(s)")
print(f"  • Total observations: {len(df)}")

if CONFIG["STATISTICAL_TESTS"]:
    n_sig = len(statistical_tests[statistical_tests["p_value"] < 0.05])
    print(f"STATISTICAL FINDINGS:")
    print(f"  • {n_sig} significant associations found (p < 0.05)")

if len(patterns) > 0:
    print(f"KEY PATTERNS:")
    print(f"  • {len(patterns)} notable differences between Q1 and Q4")
    top_3 = patterns.nlargest(3, "Difference", keep="all")
    print("\n  Top features in high-visibility content:")
    for _, row in top_3.iterrows():
        print(f"    • {row['Feature']} ({row['Case']}): +{row['Difference']:.2f}")

print("NEXT STEPS:")
print("  1. Review the Excel file for detailed results")
print("  2. Examine statistical significance in 'Statistical_Tests' sheet")
print("  3. Interpret patterns using the 'Interpretation_Guide'")
print("  4. Use heatmaps to visualise cross-quartile trends")
print("  5. Report significant findings with quartile means and p-values")
print("=" * 70)


ANALYSIS COMPLETE
SUMMARY:
  • Analysed 3 visibility metric(s)
  • Examined 9 discourse features
  • Across 7 case(s)
  • Total observations: 192
STATISTICAL FINDINGS:
  • 17 significant associations found (p < 0.05)
KEY PATTERNS:
  • 101 notable differences between Q1 and Q4

  Top features in high-visibility content:
    • Eliteness (USA): +51.60
    • Eliteness (USA): +47.20
    • Eliteness (USA): +46.20
NEXT STEPS:
  1. Review the Excel file for detailed results
  2. Examine statistical significance in 'Statistical_Tests' sheet
  3. Interpret patterns using the 'Interpretation_Guide'
  4. Use heatmaps to visualise cross-quartile trends
  5. Report significant findings with quartile means and p-values
